In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import optuna
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
df = pd.read_csv(r"C:\Users\Robyi\Documents\Data Science Dataset\delivery.csv")
df = df.drop(columns=df[['Order_ID','Weather', 'Traffic_Level','Time_of_Day', 'Vehicle_Type']])
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.head()

In [ ]:
X = df.drop(columns=['Delivery_Time_min']).values
y = df['Delivery_Time_min'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
tfd = tfp.distributions

def create_bnn(hidden_units=16, learning_rate=0.01):
    model = tf.keras.Sequential([
        tfp.layers.DenseVariational(
            units=hidden_units, 
            make_prior_fn=lambda: tfd.Normal(loc=0., scale=1.),
            make_posterior_fn=lambda: tfd.Normal(
                loc=tf.Variable(tf.random.normal([hidden_units])),
                scale=tf.nn.softplus(tf.Variable(tf.random.normal([hidden_units])))
            ),
            activation="relu"
        ),
        tfp.layers.DenseVariational(
            units=hidden_units,
            make_prior_fn=lambda: tfd.Normal(loc=0., scale=1.),
            make_posterior_fn=lambda: tfd.Normal(
                loc=tf.Variable(tf.random.normal([hidden_units])),
                scale=tf.nn.softplus(tf.Variable(tf.random.normal([hidden_units])))
            ),
            activation="relu"
        ),
        tfp.layers.DenseVariational(
            units=1,
            make_prior_fn=lambda: tfd.Normal(loc=0., scale=1.),
            make_posterior_fn=lambda: tfd.Normal(
                loc=tf.Variable(tf.random.normal([1])),
                scale=tf.nn.softplus(tf.Variable(tf.random.normal([1])))
            )
        )
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
                  loss="mse",
                  metrics=["mae"])
    
    return model

In [ ]:
def objective(trial):
    hidden_units = trial.suggest_int("hidden_units", 8, 64)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)

    model = create_bnn(hidden_units, learning_rate)
    
    history = model.fit(X_train, y_train, epochs=50, verbose=0, batch_size=16, validation_split=0.2)
    
    val_loss = history.history["val_loss"][-1]
    return val_loss

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

In [ ]:
best_params = study.best_params
print("Best Hyperparameters:", best_params)

In [ ]:
best_model = create_bnn(hidden_units=best_params["hidden_units"], learning_rate=best_params["learning_rate"])

history = best_model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=1, validation_split=0.2)

In [ ]:
y_pred_samples = np.array([best_model(X_test) for _ in range(100)])

y_pred_mean = y_pred_samples.mean(axis=0).flatten()
y_pred_std = y_pred_samples.std(axis=0).flatten()

In [ ]:
mse = mean_squared_error(y_test, y_pred_mean)
r2 = r2_score(y_test, y_pred_mean)

print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"R-squared (R²): {r2:.4f}")

In [ ]:
plt.figure(figsize=(8, 5))
plt.scatter(X_test, y_test, color="red", alpha=0.6, label="Actual Data")
plt.plot(X_test, y_pred_mean, color="blue", label="Predicted Mean")
plt.fill_between(X_test.flatten(), 
                 y_pred_mean - 2 * y_pred_std, 
                 y_pred_mean + 2 * y_pred_std, 
                 color="blue", alpha=0.3, label="Uncertainty (95%)")
plt.xlabel("X")
plt.ylabel("Y")
plt.legend()
plt.title("Bayesian Neural Network Regression with Uncertainty")
plt.show()